In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder
    .appName("Optimisation demo")
    .master("local[*]")
    .getOrCreate()
)

24/04/20 23:35:40 WARN Utils: Your hostname, pranav-Lenovo-ideapad-310-15IKB resolves to a loopback address: 127.0.1.1; using 192.168.1.10 instead (on interface wlp2s0)
24/04/20 23:35:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/20 23:35:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [7]:
import time

def get_time(func):
    def inner_get_time() -> str:
        start_time = time.time()
        func()
        end_time = time.time()
        print("-"*80)
        return (f"Execution time: {(end_time - start_time)*1000} ms")
    print(inner_get_time())
    print("-"*80)

In [6]:
print("Parallelism : ",spark.sparkContext.defaultParallelism)

Parallelism :  4


In [11]:
sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
print("sales data partitions : ",sales.rdd.getNumPartitions())

sales data partitions :  31


In [ ]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "")


In [13]:
@get_time
def x():
    sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
    print(f"Number of Partition -> {sales.rdd.getNumPartitions()}")
    sales.write.format("noop").mode("overwrite").save()

Number of Partition -> 31


24/04/20 22:08:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
Expected: _c0 but found: 
CSV file: file:///home/pranav/Desktop/projects/learning/data/flipkart/sales/fact_sales_apr1.csv
24/04/20 22:08:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_wei

--------------------------------------------------------------------------------
Execution time: 178456.2931060791 ms
--------------------------------------------------------------------------------


In [18]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "524288000")


@get_time
def x():
    sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
    # sales = sales.coalesce(8)
    print(f"Number of Partition -> {sales.rdd.getNumPartitions()}")
    sales.write.format("noop").mode("overwrite").save()


24/04/20 22:26:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
Expected: _c0 but found: 
CSV file: file:///home/pranav/Desktop/projects/learning/data/flipkart/sales/fact_sales_apr2.csv
24/04/20 22:26:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_wei

Number of Partition -> 8


24/04/20 22:27:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
Expected: _c0 but found: 
CSV file: file:///home/pranav/Desktop/projects/learning/data/flipkart/sales/fact_sales_may1.csv
24/04/20 22:27:25 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_wei

--------------------------------------------------------------------------------
Execution time: 147867.47097969055 ms
--------------------------------------------------------------------------------


In [21]:
# Disabling the AQE to manually test and optimize the performance

spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [23]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "52428800")
spark.conf.set("spark.sql.shuffle.partitions","400")

@get_time
def x():
    sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
    products = spark.read.csv("data/flipkart/products/*.csv", header= True, inferSchema = True)
    # sales = sales.coalesce(8)
    print(f"Number of Partition -> {sales.rdd.getNumPartitions()}")
    city_products = (
    sales.
    join(products, on = ["product_id"] ,how = "inner")
    # .select("city_name","product_name", "procured_quantity")
    )

    city_products.write.format("noop").mode("overwrite").save()

Number of Partition -> 80


24/04/20 22:48:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
Expected: _c0 but found: 
CSV file: file:///home/pranav/Desktop/projects/learning/data/flipkart/sales/fact_sales_apr1.csv
24/04/20 22:48:38 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_wei

--------------------------------------------------------------------------------
Execution time: 206940.043926239 ms
--------------------------------------------------------------------------------


In [12]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "52428800")
# spark.conf.set("spark.sql.shuffle.partitions","400")
spark.conf.set("spark.sql.adaptive.enabled", False)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", False)
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

import random
from pyspark.sql.functions import udf

spark.conf.set("spark.sql.shuffle.partitions", "800")

# UDF to return a random number every time and add to Employee as salt
@udf
def salt_udf():
    return random.randint(0, 800)

In [5]:
sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
sales = slaes.drop("_c0")
products = spark.read.csv("data/flipkart/products/*.csv",header = True, inferSchema = True)
producs = products.drop("_c0")
# sales.cache()

In [10]:
from pyspark.sql.functions import lit, concat

# sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
# products = spark.read.csv("data/flipkart/products/*.csv",header = True, inferSchema = True)

salted_sales = sales.withColumn("salted_product_id", concat("product_id", lit("_"), salt_udf()))

salted_products= products.withColumn("salted_product_id", concat("product_id", lit("_"), salt_udf()))


In [ ]:
# spark.conf.set("spark.sql.files.maxPartitionBytes", "52428800")
# spark.conf.set("spark.sql.shuffle.partitions","400")
from pyspark.sql.functions import broadcast

@get_time
def x():
    # sales = spark.read.csv("data/flipkart/sales/*.csv",header = True, inferSchema = True)
    # products = spark.read.csv("data/flipkart/products/*.csv", header= True, inferSchema = True)
    # sales = sales.coalesce(8)
    # print(f"Number of Partition -> {sales.rdd.getNumPartitions()}")
    city_products = (
    sales.
    join(products, on = ["product_id"] ,how = "inner")
    # .select("city_name","product_name", "procured_quantity")
    )

    city_products.write.format("noop").mode("overwrite").save()

24/04/20 23:44:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
Expected: _c0 but found: 
CSV file: file:///home/pranav/Desktop/projects/learning/data/flipkart/sales/fact_sales_apr1.csv
24/04/20 23:45:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_weighted_landing_price
 Schema: _c0, Unnamed: 0, date_, city_name, order_id, cart_id, dim_customer_key, procured_quantity, unit_selling_price, total_discount_amount, product_id, total_wei